The data were downloaded from the data asscociated with paper:

Chen, J., Sasaki, H., Lai, H. et al. Three-dimensional residual channel attention networks denoise and sharpen fluorescence microscopy image volumes. Nat Methods 18, 678–687 (2021). https://doi.org/10.1038/s41592-021-01155-x

Data can be downloaded from here: https://zenodo.org/record/4624364#.Y9bWOoHMIqJ

We will use two datasets "Confocal_2_STED.zip"

In [1]:
import pooch
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
import matplotlib.pyplot as plt
import zipfile
from pathlib import Path
from random import random
import numpy as np
from shutil import move


data_path = Path("../../data/modalityTransfer")
p = data_path / Path("download")
p.mkdir(exist_ok=True)
p = data_path / Path("train")
p.mkdir(exist_ok=True)
p = data_path / Path("test")
p.mkdir(exist_ok=True)

In [2]:
source = pooch.retrieve(
    url="doi:10.5281/zenodo.4624363/Confocal_2_STED.zip",
    known_hash="1e912c15746d8d197ddc275cc68e37764a4118d07be607f3f89751070ba75d41",
    fname="source.zip",
    path=data_path / Path("download")
)

In [3]:
with zipfile.ZipFile(source,"r") as zip_ref:
    zip_ref.extractall(data_path / Path("download"))

In [6]:
# We will use two folders in this demo: "Microtubule" and "Nuclear_Pore_complex".
# In each folder, there are two sub-folders, "raw" and "gt", while files in these two
# sub-folders are paired by filenames. Our package can support this type of setup.
# All we need is to rename the folders into "IM" and "GT". 
for ds in ["Microtubule", "Nuclear_Pore_complex"]:
    src_dir = data_path / Path("download") / Path("Confocal_2_STED") / f"{ds}" / Path("Training") / Path("raw")
    tar_dir = data_path / Path("train") / f"{ds}" / Path("IM")
    move(src_dir, tar_dir)

    src_dir = data_path / Path("download") / Path("Confocal_2_STED") / f"{ds}" / Path("Training") / Path("gt")
    tar_dir = data_path / Path("train") / f"{ds}" / Path("GT")
    move(src_dir, tar_dir)

In [4]:
# there is a bug in the training data of "Microtubule". The "gt" folder contains more images than "raw". So, we have to remove them
gt_path = data_path / Path("train") / Path("Microtubule") / Path("GT")
filenames = sorted(gt_path.glob("*.tif"))
for fn in filenames:
    raw_fn = data_path / Path("train") / Path("Microtubule") / Path("IM") / fn.name
    if not raw_fn.is_file():
        fn.unlink()

In [4]:
for ds in ["Microtubule", "Nuclear_Pore_complex"]:
    src_dir = data_path / Path("download") / Path("Confocal_2_STED") / f"{ds}" / Path("test")
    tar_dir = data_path / Path("test") / f"{ds}"
    move(src_dir, tar_dir)

In [7]:
# you may remove the download folder now.
from shutil import rmtree
rmtree(data_path / Path("download"))